# PythorchModuleStorage 
## Examples

In [17]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import numpy as np
from functools import partial, reduce
from collections import OrderedDict
from pprint import pprint
from PytorchStorage import ForwardModuleStorage

from torchvision.models import vgg16

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Forward hook

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
x = torch.rand(1,3,224,224).to(device) # random input, this can be an image
y = torch.rand(1,3,224,224).to(device) # random input, this can be an image
cnn = nn.Sequential(
    nn.Conv2d(3, 32, kernel_size=3),
    nn.MaxPool2d(2),
    nn.Conv2d(32, 64, kernel_size=3),
    nn.MaxPool2d(2),
    nn.Conv2d(64, 128, kernel_size=3),
    nn.MaxPool2d(2)
).to(device).eval()

In [19]:
storage = ForwardModuleStorage(cnn, [cnn[0], cnn[2]])
x = torch.rand(1,3,100,100).to(device)
storage([x, x])

x1 = storage[cnn[0]]

storage.clear()

In [20]:
a, b = storage[[cnn[0], cnn[2]]]
a

[tensor([[[[-3.0555e-01, -1.9099e-01, -2.5020e-01,  ..., -1.7251e-01,
            -7.8548e-02, -6.3675e-01],
           [-4.7334e-01, -5.4493e-01, -5.7556e-01,  ..., -3.7328e-01,
            -1.4060e-01, -8.9546e-01],
           [-1.9413e-01, -4.6071e-01, -6.9081e-01,  ..., -5.0491e-01,
            -1.9266e-01, -7.3968e-01],
           ...,
           [-2.2782e-01, -3.5100e-01, -5.1235e-01,  ..., -3.6148e-01,
            -3.8522e-01, -5.3316e-01],
           [-3.1731e-01, -6.0591e-01, -3.7919e-01,  ..., -8.4955e-01,
            -3.3242e-01, -4.8135e-01],
           [-2.5135e-01, -4.9734e-01, -3.7898e-01,  ..., -7.2338e-01,
            -4.8595e-01, -9.2302e-02]],
 
          [[ 5.1871e-02,  1.0453e-01, -8.3488e-03,  ..., -7.1339e-02,
            -1.9589e-01,  2.4705e-01],
           [ 5.8500e-02,  1.5596e-01, -3.5423e-02,  ..., -2.9877e-02,
            -2.2418e-01,  1.5639e-01],
           [ 5.4716e-02,  2.5099e-01,  1.9056e-01,  ...,  1.8037e-01,
            -1.5883e-02,  3.0149e-01],
